# Part 1:

In [1]:
import numpy as np                
import scipy.stats as sts           
import pandas as pd                 
import matplotlib.pyplot as plt    
import seaborn as sns  
import math 
import scipy
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
import statsmodels.api as sm

In [2]:
#Import the data
marketing_data = pd.read_csv("marketing_data.csv", header=None)

In [3]:
marketing_data

,0,1,2
0,83.689,7.015300,51.503
1,81.268,0.000000,0.000
2,78.911,0.000000,0.000
3,77.005,11.568000,0.000
4,74.772,0.000000,0.000
...,...,...,...
2438,54.604,0.000000,0.000
2439,53.049,0.000000,0.000
2440,51.521,0.000000,0.000
2441,67.118,0.000055,44.673


In [4]:
np.sum(marketing_data[0]) #sales

162861.05

In [5]:
np.sum(marketing_data[[1,2]]) #expenditures

1    13779.478384
2    12748.691232
dtype: float64

In [6]:
#rename columns
marketing_data = marketing_data.rename(columns={0:"Sales", 1:"Google", 2:"YT"}) 
marketing_data


,Sales,Google,YT
0,83.689,7.015300,51.503
1,81.268,0.000000,0.000
2,78.911,0.000000,0.000
3,77.005,11.568000,0.000
4,74.772,0.000000,0.000
...,...,...,...
2438,54.604,0.000000,0.000
2439,53.049,0.000000,0.000
2440,51.521,0.000000,0.000
2441,67.118,0.000055,44.673


In [47]:
marketing_data.Sales[1:]

1       81.268
2       78.911
3       77.005
4       74.772
5       72.614
         ...  
2438    54.604
2439    53.049
2440    51.521
2441    67.118
2442    67.359
Name: Sales, Length: 2442, dtype: float64

## Question 1

In [ ]:
#Plot Sales, Google- and YT search expenditures
marketing_data.plot(subplots=True, figsize=(10,10))
plt.show()



## Question 2

In [48]:
#Least Squares Function
def least_squares(theta, T=len(marketing_data)):
    
    #transform parameter vector theta with link functions to impose restrictions
    mu     = np.exp(theta[0])  #to ensure >0
    phi1   = np.exp(theta[1])
    phi2   = np.exp(theta[2])
    delta1 = np.exp(theta[3])/(1+np.exp(theta[3]))  #to ensure >0 and <1
    delta2 = np.exp(theta[4])/(1+np.exp(theta[4]))
    alpha1 = np.exp(theta[5])
    alpha2 = np.exp(theta[6])
    beta1  = np.exp(theta[7])/(1+np.exp(theta[7]))
    beta2  = np.exp(theta[8])/(1+np.exp(theta[8]))

#     mu, phi1, phi2, delta1, delta2, alpha1,alpha2, beta1, beta2 = theta
    
    gads   = np.zeros(T)
    yads   = np.zeros(T)
    ls_function = np.zeros(T)
    ls_function[0] = 0
    gads[0]= 0
    yads[0]= 0
  
    s  = marketing_data.Sales[1:]
    g  = marketing_data.Google
    y  = marketing_data.YT
    
    
    for t in range(1,T):
        gads[t] = beta1*gads[t-1] + alpha1*g[t]
        yads[t] = beta2*yads[t-1] + alpha2*y[t]
    
    gads = gads[1:]
    yads = yads[1:]
     
    ls_function = s - mu - phi1*(gads**delta1) - phi2*(yads**delta2)
        
    return 1/T*np.sum(ls_function[1:]**2)



In [49]:
#initial parameter values
mu_ini     = 1
phi1_ini   = 1
phi2_ini   = 1
delta1_ini = 0.5
delta2_ini = 0.5
alpha1_ini = 5
alpha2_ini = 5
beta1_ini  = 0.9
beta2_ini   = 0.9
 
#use inverse of link functions
theta_ini = np.array([np.log(mu_ini),
                      np.log(phi1_ini),
                      np.log(phi2_ini),
                      np.log(delta1_ini/(1-delta1_ini)),
                      np.log(delta2_ini/(1-delta2_ini)),
                      np.log(alpha1_ini),
                      np.log(alpha2_ini),
                      np.log(beta1_ini/(1-beta1_ini)),
                      np.log(beta2_ini/(1-beta2_ini))])
                    

# theta_ini = np.array([mu_ini, phi1_ini, phi2_ini, delta1_ini, delta2_ini, alpha1_ini,alpha2_ini, beta1_ini, beta2_ini])



                     
                                                          
                                                          
                                                           

In [53]:
#Now, optimize 
results = scipy.optimize.minimize(least_squares, theta_ini, options={'maxiter':5000},method='Nelder-Mead')
print(f'parameter estimates:')
print(f'')
print(f"mu     : {np.exp(results.x[0])}")
print(f"phi1   : {np.exp(results.x[1])}")
print(f"phi2   : {np.exp(results.x[2])}")
print(f"delta1 : {np.exp(results.x[3])/(1+np.exp(results.x[3]))}")
print(f"delta2 : {np.exp(results.x[4])/(1+np.exp(results.x[4]))}")
print(f"alpha1 : {np.exp(results.x[5])}")
print(f"alpha2 : {np.exp(results.x[6])}")
print(f"beta1  : {np.exp(results.x[7])/(1+np.exp(results.x[7]))}")
print(f"beta2  : {np.exp(results.x[8])/(1+np.exp(results.x[8]))}")

#print(results.x)
print(f'')
print(f'')

print('ls function value')
print(results.fun)
print('exit flag:')
print(results.success)
print('message:')
print(results.message)

parameter estimates:

mu     : 0.8894714468369126
phi1   : 1.0524757649670828
phi2   : 0.8198202350323597
delta1 : 0.5441863264410819
delta2 : 0.5305902881541009
alpha1 : 0.08158315572970683
alpha2 : 40.5709056972415
beta1  : 0.8967026492829732
beta2  : 0.9458904358253152


ls function value
20.297279551100452
exit flag:
True
message:
Optimization terminated successfully.


In [ ]:
#As mentioned in pdf file, small differences can produce vastly different point estimates. method to 'Neld
#Changing the optimization method to 'Nelder-Mead' for example gives very different point estimates.
#As we are not sure of the correctness of the parameters,
#We use parameter values given in PDF file for remainder of Part 1 questions

## Question 3

In [ ]:
#parameter values (given in  Assignment Instructions PDF file)
mu, phi1, phi2, delta1, delta2, alpha1, alpha2, beta1, beta2, = (0.617,2.88,19.51,0.27,0.60,0.203,0.065,0.949,0.95)

#Google and YT adstock
T=len(marketing_data)
gads   = np.zeros(T)
yads   = np.zeros(T)
gads[0]= 0
yads[0]= 0

g  = marketing_data.Google
y  = marketing_data.YT

for t in range(1,T):
        gads[t] = beta1*gads[t-1] + alpha1*g[t]
        yads[t] = beta2*yads[t-1] + alpha2*y[t]
        

plt.plot(gads, label='Google')
plt.plot(yads, label='YT ')
plt.legend()
print('Adstocks for Google and YT')
plt.show()



## Question 4

# Part 2:

## Question 9

In [ ]:
import numpy as np                
import scipy.stats as sts           
import pandas as pd                 
import matplotlib.pyplot as plt    
import seaborn as sns  
import math 
import scipy
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
import statsmodels.api as sm

In [ ]:
#Import the data
pricing_data = pd.read_csv("pricing_data.csv")
pricing_data

In [ ]:
#Plot Sales, Price, Acquistion Costs, Marketing Expenditures
pricing_data.plot(subplots=True, figsize=(12,12))
plt.show()

## Question 10

In [ ]:
#Regression of sales (S) on prices (P)
p = pricing_data['p'].values.reshape(-1, 1)
s = pricing_data['s'].values.reshape(-1, 1)

reg = LinearRegression().fit(p, s)

print(f"R-squared: {reg.score(p, s)}")
print(f"alpha: {reg.intercept_}")
print(f"beta: {reg.coef_}")


## Question 13

In [ ]:
#First, regression of prices (P) on costs (c)
c = pricing_data['c'].values.reshape(-1, 1)
p = pricing_data['p'].values.reshape(-1, 1)

reg2 = LinearRegression().fit(c, p)


print(f"alpha: {reg2.intercept_}")
print(f"beta: {reg2.coef_}")


In [ ]:
#Then, use coefficients to make p_hat
T=len(pricing_data)
p_hat   = np.zeros(T)

for t in range(0,T):
    p_hat[t] = reg2.intercept_ + reg2.coef_* pricing_data.c[t]

In [ ]:
#Now use p_hat instead of p as Question 10:

#Regression of sales (S) on predicted_prices (p_hat)
p_hat = p_hat.reshape(-1, 1)
s = pricing_data['s'].values

reg3 = LinearRegression().fit(p_hat, s)

print(f"R-squared: {reg3.score(p_hat, s)}")
print(f"alpha: {reg3.intercept_}")
print(f"beta: {reg3.coef_}") 

## Question 15

In [ ]:
#Regression of sales (S) on predicted_prices (p_hat) & marketing expenditures (m)
pricing_data['p_hat'] = p_hat
x= pricing_data[['p_hat', 'm']]
s = pricing_data['s']

reg4 = LinearRegression()
reg4.fit(x, s)

print(f"R-squared: {reg4.score(x, s)}")
print(f"alpha: {reg4.intercept_}")
print(f"beta: {reg4.coef_[0]}") 
print(f"psi: {reg4.coef_[1]}")


## Question 16

In [ ]:
T=len(pricing_data)
alpha = reg4.intercept_
beta  = reg4.coef_[0]
psi   = reg4.coef_[1]

p = pricing_data['p']
m = pricing_data['m']
c = pricing_data['c']

exp_profit = np.zeros(T)

for t in range(0,T):
    exp_profit[t] = (alpha + beta*p[t] + psi*m[t]) * (p[t]-c[t])

In [ ]:
#Lets take the profit at the last observed values in the sample. 
print(f'profit before increasing the price :  {exp_profit[T-1]}')

#If we now increase the price by one unit:
t=T-1
print(f'profit after increasing the price  :  {(alpha + beta*(p[t]+1) + psi*m[t]) * ((p[t]+1)-c[t])}')


#As profits go up by increasing the price with one unit, we advise to increase the price.